# Interrupción Legal del Embarazo

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' ##### API SÓLO TRAE INFO 2016
FILE_NAME = 'interrupcion_embarazo'

## Import

In [2]:
from mariachis.models import InterrupcionEmbarazo
ile = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)
print(ile)

Directorio: 	/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


In [3]:
##### API SÓLO TRAE INFO 2016
# ile.full_import(api=True, api_export=False, resource_id=RESOURCE_ID)['ano'].value_counts(1)

In [4]:
df = ile.full_import(api=False)
df.sample()

Archivo con nombre interrupcion_embarazo.csv fue encontrado en /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data
Con 79403 renglones y 47 columnas


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
28013,2019.0,JUNIO,2019-06-19,NaN,CASADA,28.0,NINGUNO,PREPARATORIA,TRABAJADORA DEL HOGAR NO REMUNERADA,CATOLICA,...,SI,SI,CONDON + IMPLANTE SUBDERMICO,NaN,NaN,NaN,NaN,DFSSA003640,ASPIRACIÓN ENDOUTERINA (MANUAL O ELÉCTRICA),NaN


## Diccionario de variables

### Limpiar

In [5]:
# Diccionario para aplicar limpieza por tipo de variable
clean_dict = {}

# Variables que o son 0 o > 0
clean_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']

# Numéricas que serán partidas en rangos
clean_dict['vars_num'] = {
    'edad': [9,11,15,18,21,25,29,35],
    'menarca': [7,11,12,13,14,15],
    'fsexual': [7,11,15,18,21],
    'sememb': [3,6,9,12,20],
    'nhijos': [0,1,2],
    'gesta': [0,1,2,3,4],
    'c_num': [0,1,2],
    'p_semgest': [0,5,6,7,8,9],
    'p_diasgesta': [0,1,2,3,4,5]
}

# Obtener la primer palabra
clean_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Binarias de sí o no
clean_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile']

# Limpiar y mantener para clustering posterior
clean_dict['vars_clean_keep'] = ['motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep', 'procile_simplificada', 'procile_completa']

# Sólo limpiar
clean_dict['vars_just_clean'] = ['entidad','alc_o_municipio']

# Agrupar categorías
clean_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'DESEMPLEADX',
    },
    'desc_derechohab':{
        'NINGUNO':'NINGUNO',
        'IMSS':'ALGUNO',
        'SEGURO POPULAR':'ALGUNO',
        'OTRA':'ALGUNO',
        'ISSSTE':'ALGUNO',
    },
    'nivel_edu':{
        'LICENCIATURA':'LICENCIATURA',
        'MAESTRIA':'POSGRADO',
        'DOCTORADO':'POSGRADO',
        'PREPARATORIA':'PREPARATORIA',
        'SECUNDARIA':'SECUNDARIA',
        'PRIMARIA':'PRIMARIA',
        'SIN ACCESO A LA EDUCACION FORMAL':'SIN_ACCESO',
    },
    'parentesco':{
        'PAREJA':'PAREJA',
        'EX-PAREJA':'PAREJA',
        'FAMILIAR MUJER':'FAMILIAR',
        'FAMILIAR HOMBRE':'FAMILIAR',
        'FAMILIAR SIN ESPECIFICAR':'FAMILIAR',
        'TUTOR O RESPONSABLE LEGAL':'FAMILIAR',
        'PERSONA CERCANA':'AMIGX',
        'PERSONA CERCANA MUJER':'AMIGX',
        'PERSONA CERCANA HOMBRE':'AMIGX',
        'AMIGA':'AMIGX',
        'AMIGO':'AMIGX',
        'CONOCIDA':'OTRX',
        'CONOCIDO':'OTRX',
        'RELACION LABORAL':'OTRX',
        'OTRA':'OTRX',
    },
    'religion':{
        'NINGUNA':'NINGUNA',
        'CATOLICA':'SI',
        'CRISTIANA':'SI',
        'OTRA':'SI',
        'MORMONA':'SI',
        'TESTIGA DE JEHOVA':'SI',
        'BUDISTA':'SI',
        'JUDIA':'SI',
        'ANGLICANA':'SI',
        'MUSULMANA':'SI',
    },
}

#### Auxiliar para rangos de variables numéricas

In [6]:
from pandas import qcut
from IPython.display import display

aux_dict = {}
for col in clean_dict['vars_num']:
    new_col = df[col].map(ile.clean_number).astype(float)
    orig_bins = qcut(new_col, q=6, retbins=True, duplicates='drop')[-1]
    aux_dict[col] = list(orig_bins[:-1])
display(aux_dict)

{'edad': [11.0, 20.0, 22.0, 25.0, 28.0, 32.0],
 'menarca': [0.0, 11.0, 12.0, 13.0, 14.0],
 'fsexual': [0.0, 15.0, 16.0, 17.0, 18.0, 19.0],
 'sememb': [0.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'nhijos': [0.0, 1.0, 2.0],
 'gesta': [0.0, 1.0, 2.0, 3.0, 4.0],
 'c_num': [0.0, 1.0, 2.0],
 'p_semgest': [0.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'p_diasgesta': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]}

### Crear variables

In [7]:
# Diccionario para crear variables
vars_dict = {}

# Para calcular la diferencia entre edades
vars_dict['edad_1a_menst'] = 'menarca'
vars_dict['edad_vida_sex'] = 'fsexual'
vars_dict['edad_actual'] = 'edad'

# Para comparar los métodos anticonceptivos
vars_dict['method_before'] = 'anticonceptivo'
vars_dict['method_after'] = 'panticoncep'

## Transformar

In [8]:
df, cluster_cols = ile.wrangling_ile(df, clean_dict, vars_dict, export_result=True)

Archivo: interrupcion_embarazo_clean.csv fue exportado exitosamente en: /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


In [9]:
ile.cool_print(f'\nLas {len(cluster_cols)} columnas para clustering son:\n{cluster_cols}\n\n', sleep_time=0.01)
df.sample()


Las 38 columnas para clustering son:
['panticoncep', 'desc_derechohab', 'p_consent', 'anticonceptivo', 'resultado_ile', 'procile_simplificada', 'religion', 'nivel_edu', 'procile_completa', 'desc_servicio', 'motiles', 'autoref', 'edocivil_descripcion', 'ocupacion', 'parentesco', 's_complica', 'consejeria', 'cconsejo', 'tanalgesico', 'c_dolor', 'rango_edad', 'rango_menarca', 'rango_fsexual', 'rango_sememb', 'rango_nhijos', 'rango_gesta', 'rango_c_num', 'rango_p_semgest', 'rango_p_diasgesta', 'naborto', 'npartos', 'ncesarea', 'nile', 'fsexual_vs_menarca', 'edad_vs_menarca', 'edad_vs_fsexual', 'antes_vs_despues', 'antes_vs_despues_detalle']




,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,rango_nhijos,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,rango_fsexual_vs_menarca,rango_edad_vs_menarca,rango_edad_vs_fsexual,antes_vs_despues,antes_vs_despues_detalle
43399,2016.0,AGOSTO,2016-08-09,SI,SOLTERX,21.0,DESCONOCIDO,PREPARATORIA,ESTUDIANTE,SI,...,00,01,>= 03,>= 10,00,05 a 06,09 a 12,01 a 04,DIFERENTE,"antes: CONDON, después: DIU"


## Clustering

In [10]:
df, clustering_pipeline = ile.clustering_ile(df, cluster_cols)
df.sample()

Archivo: interrupcion_embarazo_cluster.csv fue exportado exitosamente en: /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,rango_fsexual_vs_menarca,rango_edad_vs_menarca,rango_edad_vs_fsexual,antes_vs_despues,antes_vs_despues_detalle,cluster
59358,2018.0,MARZO,2018-03-22,DESCONOCIDO,UNION_LIBRE,23.0,NINGUNO,PRIMARIA,TRAB_HOGAR_NO_REMUNERADX,SI,...,04,02,01 a 05,03,05 a 06,13 a 17,05 a 08,DIFERENTE,"antes: NINGUNO, después: DESCONOCIDO",E


## Perfilamiento

In [11]:
#### CHECKPOINT

# from pathlib import Path
# from pandas import read_csv

# BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# # BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
# RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' ##### API SÓLO TRAE INFO 2016
# FILE_NAME = 'interrupcion_embarazo'

# from mariachis.models import InterrupcionEmbarazo
# ile = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)

# cluster_cols = ['panticoncep', 'desc_derechohab', 'p_consent', 'anticonceptivo', 'resultado_ile', 'procile_simplificada', 'religion', 'nivel_edu', 'procile_completa', 'desc_servicio', 'motiles', 'autoref', 'edocivil_descripcion', 'ocupacion', 'parentesco', 's_complica', 'consejeria', 'cconsejo', 'tanalgesico', 'c_dolor', 'rango_edad', 'rango_menarca', 'rango_fsexual', 'rango_sememb', 'rango_nhijos', 'rango_gesta', 'rango_c_num', 'rango_p_semgest', 'rango_p_diasgesta', 'naborto', 'npartos', 'ncesarea', 'nile', 'fsexual_vs_menarca', 'edad_vs_menarca', 'edad_vs_fsexual', 'antes_vs_despues', 'antes_vs_despues_detalle']

# df = read_csv(Path(BASE_DIR).joinpath(f'{FILE_NAME}_cluster.csv'), low_memory=False)
# df.sample()

In [12]:
add_cols = ['fingreso_yearmonth','entidad','alc_o_municipio']
ile.profiles(df[add_cols+cluster_cols+['cluster']])